<a href="https://colab.research.google.com/github/bodorcy/hazifeladatok/blob/main/ml_4_feature_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Szöveges dokumentumok osztályozása
Olvasd el a [szövegbányászat](https://inf.u-szeged.hu/~rfarkas/ML20/NLP.html) előadás olvasóleckét.

Építsünk egy osztályozó gépi tanuló algoritmust, amit tanítunk, majd képes lesz korábban nem látott szövegekről döntést hozni, hogy azok pozitív, negatív vagy semleges véleményt fejeznek-e ki!

Töltsük le a tanító adatbázist:

In [ ]:
import pandas as pd
train_data = pd.read_csv('https://github.com/rfarkas/student_data/raw/main/sentiment/train.tsv')
train_data

Hibaüzent :( Mi a hiba?

Ha a fájlt megnyitjuk akokr látjuk, hogy 3 oszlopot tartalmaz tabbal elválasztva (ezért tsv a kiterjesztés). A pandas.read_csv alapesetben vesszővel elválasztva várja a mezőket.

'Expected 2 fields in line 10, saw 4' Az első sor alapján a read_csv úgy gondolta, hogy két oszlop lesz, de a 10. sorban 4 oszlopot talált mert ott vesszők szerepeltek a szövegben...

**Érdemes rápillantani egy input filera mielőtt feldolgozni kezdjük!**

In [ ]:
# ha a read_csv-nek megadjuk, hogy tab a separator akkor minden helyesen működik
train_data = pd.read_csv('https://github.com/rfarkas/student_data/raw/main/sentiment/train.tsv', sep='\t')
train_data

Értsük meg mi van az adatbázisban! Például milyen címkék (label) találhatóak benne?

In [ ]:
train_data.label.hist()

## Szózsák jellemzőtér

A dokumentumok lesznek a gépi tanulásban az egyedek. A legegyszerűbb jellemzőkészket amivel reprezentálni lehet egy dokumentumot az az egyes dokumentumban előforduló szavak gyakorisága. Minden az adatbázisban előforduló szóra felveszünk egy jellemzőt. Tfh az 'alma' szó szerepel a szótárunkban. Ekkor ha a dokumentumban kétszer fordul elő, akkor a jellemző értéke 2 lesz, ha pedig nem fordul elő akkor 0.

Ezt **szózsák reprezentációnak**nek (bag-of-words) is hívják, mivel olyan mintha a szavakat beöntenénk egy zsákba, elveszik azok sorrendisége (pl. nem mindegy 'not' és 'good' szavak egymás után fordultak-e elő) és pozíciója. De első gépi tanulási kísérletre pont jó :)

Az `sklearn`ben a szózsákot a `CountVectorizer` jelemzőkinyerő implementálja. Ez leszámolja a dokumentumok szavait és azokat `sklearn` jellemzőkké (feature) alakítja.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [ ]:
# fit: összegyűjti a szókincset, azaz minden tokent ami legalább egyszer előfordul az adatbázisban
vectorizer.fit(train_data.text)

In [ ]:
len(vectorizer.get_feature_names_out())

In [ ]:
vectorizer.vocabulary_ #ua, de itt a "szókincs" a jellemzőtér

In [ ]:
# transform: végrehajtja a jellemzőkinyerést, azaz minden dokumentumhoz leszámolja minden szótárbeli szó gyakoriságát
features = vectorizer.transform(train_data.text)
features

A tanító adatbázisunk így 9063 egyedet tartalmaz (sorok) és 24285 jellemzőt (oszlopot). Azaz 24285 különböző token alkotja a szótárat. Mivel egy dokumentumban nagyon kevés szó fordul elő a 24285 szóból, ezért a jellemző mátrix túlnyomó része 0, ritka mátrixot használunk.

In [ ]:
# fit_transform: a fit és transform egymás után futtatva
features = vectorizer.fit_transform(train_data.text)
features

## Lineáris gépek

**Tanítunk** egy ún. [lineáris gép osztályozó](https://inf.u-szeged.hu/~rfarkas/ML20/linearis_gep.html) modellt, a [stochastic gradient descend (SGD)](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html). Itt az elöző órán megismert döntési fát nem szerencsés használni mert nagyon sok jellemzőnk van és azok között megtanulni a kapcsolatot nagyon sok idő. Ilyenkor [lineáris osztályozókat](https://scikit-learn.org/stable/modules/linear_model.html) (SGD mellett például Logisztikus Regresszió) érdemes használni vagy olyan döntési fa variánsokat amik sok jellemzőre lettek kitalálva (pl. [xgboost](https://xgboost.readthedocs.io/)).

In [ ]:
from sklearn.linear_model import SGDClassifier

cls = SGDClassifier()
model = cls.fit(features, train_data.label)

In [ ]:
model.coef_.shape

Az egyes osztályok legerősebb jellemzői:

In [ ]:
sorted(zip(model.coef_[2], vectorizer.get_feature_names_out()),reverse=False)[:10]

**Vizualizáljuk** a legerősebb jellemzőket!

In [ ]:
import numpy as np

In [ ]:
feature_names = vectorizer.get_feature_names_out()
feature_weights = cls.coef_

In [ ]:
rows = np.concatenate([[feature_names], feature_weights]).T
columns = ["words"] + cls.classes_.tolist()

In [ ]:
feature_df = pd.DataFrame(rows, columns=columns)
feature_df

In [ ]:
for class_label in cls.classes_:
  feature_df.sort_values(class_label, ascending=False)[:10].plot(kind="bar", x="words", y=class_label)

**Értékeljük ki**, hogy milyen pontos modellt tanítottunk ezen az adatbázison!

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
prediction = model.predict(features)
prediction # a predikció eredménye egy lista az egyedekre predikált címkékkel

In [ ]:
accuracy_score(y_true=train_data.label, y_pred=prediction)

In [ ]:
print(classification_report(y_true=train_data.label, y_pred=prediction))

Megjegyzés: a tanító adatbázison való kiértékelésnek is van haszna. Ha nagyon alacsony értékeket kapunk az azt jelenti, hogy vagy jellemzőkészlet amit kinyertünk nem tartalmaz a tanuláshoz elég információt vagy a gépi tanuló modellt rosszul választottuk meg.

In [ ]:
test_data = pd.read_csv('https://raw.githubusercontent.com/rfarkas/student_data/main/sentiment/test.tsv', sep='\t')

In [ ]:
vectorizer2 = CountVectorizer()
test_features = vectorizer2.fit_transform(test_data.text)
prediction = model.predict(test_features)

Hibaüzenet :( Már megint...

`'X has 7271 features per sample; expecting 24285'`

Emlékezzünk vissza, hogy a `vectorizer.fit_transform` `fit`je először az adatbázisból elkészíti a szótárat (legalább egyszer előforduló szavak az egész adatbázisban). A test adatbázisban 7271 különböző szó fordult elő. Így a test adatbázis jellemzőkészlete (`test_features`) nem kompatibilis a `model` jellemzőkészletével ami 24285 dimenziós. Fontos, hogy a tanító adatbázison kialakított jellemzőkészletet használjuk a test adatbázison is!

In [ ]:
test_features = vectorizer.transform(test_data.text)
prediction = model.predict(test_features)

In [ ]:
test_features

In [ ]:
accuracy_score(y_true=test_data.label, y_pred=prediction)

Jelen esetben a konstans baseline accuracy-ja 0.333 lenne, úgyhogy kijelenthetjük, hogy a modell tanult valamit :)

De azért a baseline-t mérjük ki hivatalosan is:

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent") # tanító adatbázis leggyakoribb osztálya lesz mindig a predikció
dummy_clf.fit(features, train_data.label) # ugyanazon a tanító adatbázison "tanítjuk"
baseline_prediction = dummy_clf.predict(test_features) # predikció a kiértékelő adatbázison
accuracy_score(baseline_prediction, test_data.label)

In [ ]:
## Az SGDClassifier-ben a loss és az alpha beállításával tudjuk elkerülni a túltanulást
## Ezt hívjuk regularizációnak lineáris gépeknél
cls = SGDClassifier(alpha=0.001)
model = cls.fit(features, train_data.label)
prediction = model.predict(test_features)
accuracy_score(y_true=test_data.label, y_pred=prediction)

In [ ]:
#egy másik lineáris gép osztályozó
from sklearn.linear_model import LogisticRegression
cls = LogisticRegression()

## Tévesztési mátrix

In [ ]:
print(classification_report(y_true=test_data.label, y_pred=prediction))

Látszik, hogy a negatív osztályt valamiért könnyebben megtanulja, mind a precision, mind a recall jobb ott, mint a másik két osztályon...

A tévesztési (confusion) mátrix megmutatja, hogy melyik osztályt melyik másik osztállyal téveszti össze a modell:

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true=test_data.label, y_pred=prediction)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
cm = confusion_matrix(y_true=test_data.label, y_pred=prediction)
ConfusionMatrixDisplay(cm, display_labels=model.classes_).plot(values_format='.3g')

A pozitív és semleges osztályokat valamiért sokszor összekeveri... Nézzünk rá példákat a tanító adatbázison!

# Modell javítása új jellemzőkészlettel

Ha a szavak mellett szópárok (bigram) is jellemzők lennének, több információt adunk át a modellnek. Így például a 'not good' megjelenik jellemzőként és esélyt adunk a modellnek, hogy összefüggést tanuljon rá.

In [ ]:
# CountVectorizernek átparaméterezésével új jellemzőkinyerést valósítunk meg, minden más ugyanaz marad
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=2)
# ngram_range mondja meg, hogy szavakat (unigram, 1gram) és egymás utáni szavakból álló párokat (bigram, 2gram) használjunk
# min_df=2 eldobja azokat a szavakat amik kevesebb, mint 2 dokumentumban fordult elő. Nagyon sok bigram van, ezzel csökkentjük a jellemzőkészlet dimenziószámát
# CountVectorizernek számos paramétere van, lásd: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

features_bigram = bigram_vectorizer.fit_transform(train_data.text)

cls = SGDClassifier(alpha=0.001) # új, üres osztályozó
model_bigram = cls.fit(features_bigram, train_data.label)
features_bigram

In [ ]:
bigram_vectorizer.vocabulary_

In [ ]:
prediction_bigram = model_bigram.predict(bigram_vectorizer.transform(test_data.text))

In [ ]:
print(accuracy_score(y_true=test_data.label, y_pred=prediction_bigram))
print(classification_report(y_true=test_data.label, y_pred=prediction_bigram))

Vannak olyan szavak amelyek sokszor fordulnak elő szövegekben, de szétválasztó erejük nincs, azaz ugyanúgy előfordulnak minden osztályban. Ilyenek például, sok más közt, a névelők. Ezek félreviszik az osztályozást (zajra tanulunk rá). A legegyszerűbb technika ennek kiküszöbölésére az ha a sima szógyakoriság (term frequency, TF) helyett normalizáljunk a szavak dokumentumok feletti gyakoriságávl (inverse document frequency, IDF). Lásd:  [TfIdf](http://www.tfidf.com)

In [ ]:
# TfidfTransformer is egy új ellemzőkinyersi mód, minden más változatlan
from sklearn.feature_extraction.text import TfidfTransformer
vectorizer = CountVectorizer()
cv_counts = vectorizer.fit_transform(train_data.text)
idf_transformer = TfidfTransformer(use_idf=True).fit(cv_counts)
features_idf = idf_transformer.transform(cv_counts)
features_idf

In [ ]:
cls = SGDClassifier()
model_idf = cls.fit(features_idf, train_data.label)

In [ ]:
prediction_idf = model_idf.predict(idf_transformer.transform(vectorizer.transform(test_data.text)))

In [ ]:
print(accuracy_score(y_true=test_data.label, y_pred=prediction_idf))
print(classification_report(y_true=test_data.label, y_pred=prediction_idf))

Itt már kézzel fogható javulást tudtunk elérni.

# Model javítása előfeldolgozással

In [ ]:
def preprocess(textcol):
    return textcol.replace('\d+', 'NUM',regex=True)

In [ ]:
train_data.text = preprocess(train_data.text)
test_data.text  = preprocess(test_data.text)

In [ ]:
train_data.text

Az [NLTK](https://www.nltk.org/) az egyik leggyakrabban használt python csomag szövegfeldolgozásban. A másik a [spaCy](https://spacy.io/). Kettejük egy összehasonlítása [itt](https://www.activestate.com/blog/natural-language-processing-nltk-vs-spacy/)

In [ ]:
import nltk

Miért nem egyértelmű a tokenizálás? Miért léteznek különböző algoritmusok?

Azért mert különböző nyelveken másképp lehetnek a szóhatárok (magyar "-e"), például rövidítést jelentő pont a token része ("U.S.A." vagy "kft."), míg mondatvégi a írásjel nem. Továbbá a szöveg típusa is megkövetelhet különböző tokenizálókat, pl. szociális médiában az emotikonok és URLeket egyben kell tartni de a camelcase szavakat (pl. JoMunkahozIdoKell) tokenizáljuk, kémiai szövegekben a kötőjel egy molekula nevében nem szóhatár, stb.

In [ ]:
text = """What can I say about this place. The staff of the restaurant is nice and the eggplant is not bad. Apart from that, very uninspired food, lack of atmosphere and too expensive. I am a staunch vegetarian and was sorely dissapointed with the veggie options on the menu. Will be the last time I visit, I recommend others to avoid."""

nltk_splitter = nltk.data.load('tokenizers/punkt/english.pickle') # Számos szöveget mondatra bontó algoritmus (splitter) van implementálva az NLTKban. A Punkt az egyik, ezt betöltjük.
nltk_tokenizer = nltk.tokenize.TreebankWordTokenizer() # Számos szavakra (tokenekre) bontó algoritmus (tokenizer) is implementálva van.

# text mondatai:
sentences = nltk_splitter.tokenize(text)

Sajnos ez így még nem működik. Az NLTK erőforrásfájlait külön le kell töltenünk a Colab-ba. Ha azok nincsenek ott hibaüzenettel elszállunk futtatás közben.

In [ ]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

Most már működik! Futassuk újra a nltk_splitter inicializáló kódcellát!

In [ ]:
sentences = nltk_splitter.tokenize(text) # a szöveget mondatokra bontjuk
sentences # mondatokat tartalmazó lista

In [ ]:
tokenized_sentences = [nltk_tokenizer.tokenize(sent) for sent in sentences] # a mondatokat bejárjuk a for ciklussal és egyessével tokenizáljuk őket
tokenized_sentences # listák listája, amiben a mondatok szavai vannak

Egy másik fontos előfeldolgozási lépés a **szótövesítés (stemming, lemmatizáció)**. A cél, hogy a szavak különböző ragozott alakjait össze tudjuk vonni (pl. általában nem érdemes külön kezelni az 'asztalaitokra' és 'asztal' szóalakokat). A lemmatizáció az igazi nyelvtani értelemben vett szótő meghatározását jelenti. Ez nagyon bonyolult feladat tud lenni bizonyos nyelveken, pl. a magyarban ahol tőhangváltás is van (a 'madarak' szó szótöve a 'madár'). De sokszor elég a szótőnek egy "közelítése", azaz egyszerű szabályokkal lecseréljük a szóalak végi karaktereket más karakterre. Ezt a közelítő (butább, de sokkal egyszerűbb és gyorsabb) hívjuk stemmelésnek ([részletesen](https://www.datacamp.com/community/tutorials/stemming-lemmatization-python))

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer # egy angol igazi lemmatizáló
nltk.download('wordnet') # erőforrást itt is le kell tölteni hozzá
lemmatizer = WordNetLemmatizer()

In [ ]:
lemmatizer.lemmatize("companies")

In [ ]:
from nltk.stem import PorterStemmer # egy angol gyors stemmer (nincs erőforrásfájl, a szabályok a kódban vannk)
stemmer = PorterStemmer()

In [ ]:
stemmer.stem("companies") # 'es' végződés levágása sokszor működik többesszámú főneveknél. A 'companies' esetén "buta".

# Gyakorló feladatok

Az órai adatbázison hajts végre egy kísérletet (tanítás, predikció és kiértékelés) ahol

*   a szavak szótövét vagy stemjét használjuk a szózsák modellben!
*   egy másik lineáris gépet, a Logisztkus Regresszió osztályozó algoritmust használunk (Logistic Regression Classifier).

Írd ki, hogy mekkora szótár lesz így illetve mennyi így az accuracy!

---
**Következő két hétben:**
Az elmúlt 10+ évben a szózsák modellt felváltották a szóbeágyazás alapú reprezentációk, pl. [word2vec](https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial) illetve az ezeket használó deep learning osztályozók, konvolúciós és rekurrens neurális hálózatok. Aztán 2019 óta nagy nyelvi modellek nyertek teret, mint a BERT és a GPT